### Lab 3

In [17]:
#Import modules
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [18]:
df = pd.read_csv('./data/titanic.csv', sep=',')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
df.isna().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64



Fill Age, Fare and Embarked with sensible values. (Embarked could be filled with "S")
Since the nan values are defined differently in this dataset, we can use the function right out of the box.

Don't do anything with the Survived column for now.

During the last lab we showed how to fill nan values with meaningful values. What a meaningful value is differ from dataset to dataset, in this dataset the age value is right skewed, so we will use the median.

We can check the numerical difference of mean and median with:


In [20]:
# mean age
print('Mean of "Age" is %.2f' %(df["Age"].mean(skipna=True)))
# median age
print('Median of "Age" is %.2f' %(df["Age"].median(skipna=True)))

Mean of "Age" is 29.88
Median of "Age" is 28.00


In [21]:
# Replace NaN values in "Age" with the median value
df["Age"] = df["Age"].fillna(df["Age"].median())
df.isna().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

We can also see that many people in the dataset have a NaN value for Cabin. It’s not as easy as just fill a dummy value here. Since almost 77% of the data is missing here we will assume that the missing data indicates that they had no cabin. We could fill with “no cabin”, but for machine learning, we like to have numerical or bool values. To achieve this, lets make a new bool column:

Cabin = True / False
And set all NaN values = False, all other = True
```python
df["HasCabin"] = df.Cabin.notnull()
# or alternatively: df["HasCabin"] = ~df.Cabin.isnull()
```
do a df.head() and you can see we have a new column. Note: We want True for passengers who HAVE a cabin, so we use notnull() rather than isnull(). 

Adding a new column based on data available is considered creating a new feature.

**Another viable approach** in this scenario would be to remove the entire column from the dataset. The presence of approximately 77% missing values in the column is considered highly detrimental to the overall quality and integrity of the dataset.

In [22]:
df["hasCabin"] = df.Cabin.notnull().astype('int')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,hasCabin
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [23]:
df = df.drop(columns=['Cabin','hasCabin'])
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


**Which did you do and why?**

I deleted the column, because of the missing 77%  of values. I think the value might be of value, but i also want less clutter doing this lab. So i assume since you give the option it's okay to delete it.

**2. Adding a feature**

Lets extract the title for each person on the boat, and make a new column called «Title»
As we can see from the data set, the syntax for names is
LastName, Title. RestOfName

![names][names]
An easy way to extract a sertan string is to use:
```python
lambda x: re.search(' ([A-Z][a-z]+)\.', x).group(1)
```

*What is this syntax? It's called regex, and a [explanation can be found here][regex]*

In this scenario we are looking for groups of letters A-Z or a-z that end with a dot (.), which we then put into groups.

And in our case we would like to put this data in a new column, so we can run 
```python
df["Title"] = df.Name.apply(lambda x: re.search(' ([A-Z][a-z]+)\.', x).group(1)) 
```

Check with df.head() that you now have a column called Title.
We can now see how many has each title. This can be done in many ways, but calling 
```python
df["column"].value_counts() # you need to replace "column" 
# with the name of the column you want to count
```
![count][cC]

As we can see from the count, we have 18 titles, some of them with only one person. 
Replace Mlle and Ms with "Miss", and Mme with "Mrs" using:
```python
df["column"] = df["column"].replace({'xxx':'yyy', 'jjj':'iiii', … 'uuu':'iii'})
```

In [26]:
df["Title"] = df.Name.apply(lambda x: re.search(r' ([A-Z][a-z]+)\.', x).group(1))
# re.search(' ([A-Za-z]+)\.', x) finds the title in the name string, + means one or more characters . means a dot, group(1) means the first group in the regex

df['Title'] = df['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'}, regex=True)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,Mr
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,Mrs
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,Miss
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,Mrs
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,Mr


We can also package all titles with few persons into a unique category
```python
df["column"] = df["column"].replace(["x","y", … , "n"], "Unique")
```
And do a new count of titels and see if you get something simellare to this:
![recount][cC2]

You can also produce a plot with
```python
sns.countplot(data=df, x='Title')  # Seaborn countplot
plt.xticks(rotation=45);
```
![plot][pl1]

In [27]:
df['Title'].value_counts()


Title
Mr          757
Miss        264
Mrs         198
Master       61
Rev           8
Dr            8
Col           4
Major         2
Don           1
Lady          1
Sir           1
Capt          1
Countess      1
Jonkheer      1
Dona          1
Name: count, dtype: int64

In [31]:
# using[x,y,z] gave ma a warning about tuples. So gpt suggested this:
df['Title'] = df['Title'].replace({r'Lady|Don|Major|Col|Dr|Rev|Master|Sir|Capt|Countess|Jonkheer|Uniquea':'Unique'}, regex=True)
df['Title'].value_counts()

Title
Mr        757
Miss      264
Mrs       198
Unique     90
Name: count, dtype: int64

**3. Convert Age and Fare into categorical data.**

 This can be done using pandas qcut function
```python
df['CatAge'] = pd.qcut(df["Age"], q=4, labels=False )
```
do this for both Age and Fare.

****

In [33]:
df['CatAge'] = pd.qcut(df['Age'], 4, labels=['Child', 'YoungAdult', 'Adult', 'Senior'])
df['CatAge'].value_counts()

CatAge
YoungAdult    466
Child         333
Senior        322
Adult         188
Name: count, dtype: int64

In [34]:
df['CatFare'] = pd.qcut(df['Fare'], 4, labels=['Low', 'Medium', 'High', 'VeryHigh'])
df['CatFare'].value_counts()

CatFare
Low         337
High        328
VeryHigh    323
Medium      320
Name: count, dtype: int64

**4. Convert dataframe to binary data**

In order to use this dataset for ML training, we want all data to be numerical. To achieve this, we need to drop columns that don’t make sense converting to a numerical value. At this point, your dataframe should look something like this:

![dataframe][table-task4]

Identify columns that we need to drop to convert to a numerical dataset.

[Solution][table-task4-m]

Drop the columns you identified with

```python
df = df.drop(["column1", ... , "columnN"], axis=1)
```
Converting to binary  data is a trivial task in pandas. Try using [pd.get_dummies][get-dummies]

This works well for analytic tasks, but you could also use [OneHotEncoder() for machine learning tasks.][get-dummies-vs-onehot]

In [37]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title,CatAge,CatFare
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,Mr,Child,Low
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,Mrs,Senior,VeryHigh
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,Miss,YoungAdult,Medium
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,Mrs,Adult,VeryHigh
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,Mr,Adult,Medium


In [36]:
print(list(df.columns))

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked', 'Title', 'CatAge', 'CatFare']


In [39]:
df = df.drop(columns=['PassengerId', 'Name', 'Ticket'])

In [40]:
print(list(df.columns))

['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title', 'CatAge', 'CatFare']


**5. Save the cleaned dataset to a new .csv file. We will use this in the next lab.**

*hint:*
```python
df.to_csv('data/Titanic_Cleaned.csv', index=False)
```
****
**All done**

In [41]:
df.to_csv('data/Titanic_cleaned.csv', index=False)
